In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2
from fastai.structured import *
from fastai.column_data import *
from fastai.imports import *
from fastai.dataset import *
from fastai.torch_imports import *
from fastai.column_data import *
from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import LabelEncoder , Imputer , StandardScaler
import operator
import os
np.set_printoptions(threshold=50,edgeitems=20)
PATH='data/airbnb/'

In [ ]:
!ls data/airbnb


In [ ]:
table_names = ['train_users_2','sessions']
tables = [pd.read_csv(f'{PATH}{fname}.csv',low_memory=False) for fname in table_names]

In [ ]:
test = pd.read_csv(f'{PATH}test_users.csv')

In [ ]:
for t in tables : display(t.head())

In [ ]:
for t in tables : display(DataFrameSummary(t).summary())

In [ ]:
#loading data 1.train
df_train = pd.read_csv(f'{PATH}train_users_2.csv')
target = df_train['country_destination']
df_train = df_train.drop(['country_destination'],axis=1)

In [ ]:
#    2.test
df_test = pd.read_csv(f'{PATH}test_users.csv')
id_test = df_test['id']

In [ ]:
#    3.sessions
df_sessions = pd.read_csv(f'{PATH}sessions.csv')
df_sessions['id'] = df_sessions['user_id']
df_sessions = df_sessions.drop(['user_id'],axis = 1)

preaprring session data

In [ ]:
#fillinf nan with specific value NAN
df_sessions.action = df_sessions.action.fillna('NAN')
df_sessions.action_type = df_sessions.action_type.fillna('NAN')
df_sessions.action_detail = df_sessions.action_detail.fillna('NAN')
df_sessions.device_type = df_sessions.device_type.fillna('NAN')

In [ ]:
#action values with low frequency are changed to other
act_freq = 100  #Threshold for frequency
act = dict(zip(*np.unique(df_sessions.action, return_counts=True)))
df_sessions.action = df_sessions.action.apply(lambda x: 'OTHER' if act[x] < act_freq else x)

In [ ]:
#Computing value_counts. use for one-hot encoding
f_act = df_sessions.action.value_counts().argsort()
f_act_detail = df_sessions.action_detail.value_counts().argsort()
f_act_type = df_sessions.action_type.value_counts().argsort()
f_dev_type = df_sessions.device_type.value_counts().argsort()

In [ ]:
#grouping by session id
dgr_sess = df_sessions.groupby(['id'])

In [ ]:
#loop to create features
samples = []
cont = 0
ln = len(dgr_sess)
for g in dgr_sess:
    gr = g[1]
    l = []
    l.append(g[0])  #id
    l.append(len(gr))  #number of values
    sev = gr.secs_elapsed.fillna(0).values
    #action features   how many times each value occurs, numb of unique values, mean and std
    c_act = [0] * len(f_act)
    for i,v in enumerate(gr.action.values):
        c_act[f_act[v]] += 1
    _, c_act_uqc = np.unique(gr.action.values,return_counts = True)
    c_act += [len(c_act_uqc),np.mean(c_act_uqc),np.std(c_act_uqc)]
    l = l + c_act
    
    #action_detail features
    c_act_detail = [0] * len(f_act_detail)
    for i,v in enumerate(gr.action_detail.values):
        c_act_detail[f_act_detail[v]] += 1 
    _, c_act_det_uqc = np.unique(gr.action_detail.values, return_counts=True)
    c_act_detail += [len(c_act_det_uqc), np.mean(c_act_det_uqc), np.std(c_act_det_uqc)]
    l = l + c_act_detail
    
    #action_type features with log of the sum of secs_elapsed for each value
    l_act_type = [0] * len(f_act_type)
    c_act_type = [0] * len(f_act_type)
    for i,v in enumerate(gr.action_type.values):
        l_act_type[f_act_type[v]] += sev[i]   
        c_act_type[f_act_type[v]] += 1  
    l_act_type = np.log(1 + np.array(l_act_type)).tolist()
    _, c_act_type_uqc = np.unique(gr.action_type.values, return_counts=True)
    c_act_type += [len(c_act_type_uqc), np.mean(c_act_type_uqc), np.std(c_act_type_uqc)]
    l = l + c_act_type + l_act_type 
    
    #device_type features
    c_dev_type  = [0] * len(f_dev_type)
    for i,v in enumerate(gr.device_type .values):
        c_dev_type[f_dev_type[v]] += 1 
    c_dev_type.append(len(np.unique(gr.device_type.values)))
    _, c_dev_type_uqc = np.unique(gr.device_type.values, return_counts=True)
    c_dev_type += [len(c_dev_type_uqc), np.mean(c_dev_type_uqc), np.std(c_dev_type_uqc)]        
    l = l + c_dev_type 
    
    #secs_elapsed features
    l_secs = [0] * 5 
    l_log = [0] * 15
    if len(sev) > 0:
        #Simple statistics about the secs_elapsed values.
        l_secs[0] = np.log(1 + np.sum(sev))
        l_secs[1] = np.log(1 + np.mean(sev)) 
        l_secs[2] = np.log(1 + np.std(sev))
        l_secs[3] = np.log(1 + np.median(sev))
        l_secs[4] = l_secs[0] / float(l[1])
        
        #Values are grouped in 15 intervals. Compute the number of values
        #in each interval.
        log_sev = np.log(1 + sev).astype(int)
        l_log = np.bincount(log_sev, minlength=15).tolist()                      
    l = l + l_secs + l_log
    
    samples.append(l)
    cont +=1

In [ ]:
#creating datafram
col_names = []
for i in range(len(samples[0])-1):
    col_names.append('c_'+str(i))
#preparring objects
samples = np.array(samples)
samp_ar = samples[:,1:].astype(np.float16)
samp_id = samples[:,0]

df_agg_sess = pd.DataFrame(samp_ar,columns=col_names)
df_agg_sess['id']=samp_id
df_agg_sess.index = df_agg_sess.id

working on train and test data

In [ ]:
#concatenate
df_tt = pd.concat((df_train, df_test), axis=0, ignore_index=True)
df_tt.index = df_tt.id
#replace unknown or missing values
df_tt = df_tt.fillna(-1)
df_tt = df_tt.replace('-unknown-', -1)

In [ ]:
from datetime import datetime , date

In [ ]:
df_tt['n_null'] = np.array([sum(r==-1) for r in df_tt.values])  #number of nulls

#date_account_created compute year month day week_number weekday
dac = np.vstack(df_tt.date_account_created.astype(str).apply(lambda x: list(map(int, x.split('-')))).values)
df_tt['dac_y'] = dac[:,0]
df_tt['dac_m'] = dac[:,1]
df_tt['dac_d'] = dac[:,2]
dac_dates = [datetime(x[0],x[1],x[2]) for x in dac]
df_tt['dac_wn'] = np.array([d.isocalendar()[1] for d in dac_dates])
df_tt['dac_w'] = np.array([d.weekday() for d in dac_dates])
df_tt_wd = pd.get_dummies(df_tt.dac_w, prefix='dac_w')
df_tt = df_tt.drop(['date_account_created', 'dac_w'], axis=1)
df_tt = pd.concat((df_tt, df_tt_wd), axis=1)

#time_stamp_first_active
tfa = np.vstack(df_tt.timestamp_first_active.astype(str).apply(lambda x: list(map(int, [x[:4],x[4:6],x[6:8],x[8:10],x[10:12],x[12:14]]))).values)
df_tt['tfa_y'] = tfa[:,0]
df_tt['tfa_m'] = tfa[:,1]
df_tt['tfa_d'] = tfa[:,2]
df_tt['tfa_h'] = tfa[:,3]
tfa_dates = [datetime(x[0],x[1],x[2],x[3],x[4],x[5]) for x in tfa]
df_tt['tfa_wn'] = np.array([d.isocalendar()[1] for d in tfa_dates])
df_tt['tfa_w'] = np.array([d.weekday() for d in tfa_dates])
df_tt_wd = pd.get_dummies(df_tt.tfa_w, prefix='tfa_w')
df_tt = df_tt.drop(['timestamp_first_active', 'tfa_w'], axis=1)
df_tt = pd.concat((df_tt, df_tt_wd), axis=1)

#timespans between dates
df_tt['dac_tfa_secs'] = np.array([np.log(1+abs((dac_dates[i]-tfa_dates[i]).total_seconds())) for i in range(len(dac_dates))])
df_tt['sig_dac_tfa'] = np.array([np.sign((dac_dates[i]-tfa_dates[i]).total_seconds()) for i in range(len(dac_dates))])

#age
av = df_tt.age.values
av = np.where(np.logical_and(av<2000, av>1900), 2014-av, av) #This are birthdays instead of age (estimating age by doing 2014 - value)
av = np.where(np.logical_and(av<14, av>0), 4, av) #Using specific value=4 for age values below 14
av = np.where(np.logical_and(av<2016, av>2010), 9, av) #This is the current year insted of age (using specific value = 9)
av = np.where(av > 99, 110, av)  #Using specific value=110 for age values above 99
df_tt['age'] = av

#one hot encoding features
ohe_feats = ['gender', 'signup_method', 'signup_flow', 'language', 'affiliate_channel', 'affiliate_provider', 'first_affiliate_tracked', 'signup_app', 'first_device_type', 'first_browser']
for f in ohe_feats:
    df_tt_dummy = pd.get_dummies(df_tt[f], prefix=f)
    df_tt = df_tt.drop([f], axis=1)
    df_tt = pd.concat((df_tt, df_tt_dummy), axis=1)

merge train -test and sessions

In [ ]:
df_all = pd.merge(df_tt, df_agg_sess, how='left')
df_all = df_all.drop(['id'], axis=1)
df_all = df_all.fillna(-2)
df_all['all_null'] = np.array([sum(r<0) for r in df_all.values])   #for all nulls

computing test and train values

In [ ]:
piv_train = len(target)   #marker
vals = df_all.values
le = LabelEncoder()

x = vals[:piv_train]
y = le.fit_transform(target.values)
x_test = vals[piv_train:]

In [2]:
train_users = pd.read_csv(f'{PATH}train_users_2.csv')
test_users = pd.read_csv(f'{PATH}test_users.csv')
sessions = pd.read_csv(f'{PATH}sessions.csv')

users = pd.concat([train_users, test_users], axis=0, ignore_index=True)


users.drop('date_first_booking', axis=1, inplace=True)

user_with_year_age_mask = users['age'] > 1900
users.loc[user_with_year_age_mask, 'age'] = 2015 - users.loc[user_with_year_age_mask, 'age']

users.loc[(users['age'] > 120) | (users['age'] < 18), 'age'] = -1
users['age'].fillna(-1, inplace=True)

bins = [-1, 20, 25, 30, 40, 50, 60, 75, 100, 120]
users['age_group'] = np.digitize(users['age'], bins, right=True)


list_browsers=["Arora","Avant Browser","Camino","CometBird","Comodo Dragon","Conkeror","CoolNovo","Crazy Browser","Epic","Flock","Google Earth","Googlebot","IBrowse","IceDragon","IceWeasel","Iron","Kindle Browser","Maxthon","Nintendo Browser","NetNewsWire","OmniWeb","Outlook 2007","Pale Moon","Palm Pre web browser","PS Vita browser","RockMelt","SeaMonkey","SiteKiosk","SlimBrowser","Sogou Explorer","Stainless","TenFourFox","TheWorld Browser","UC Browser","wOSBrowser","Yandex.Browser"]
for i in range(len(list_browsers)):
    users.loc[(users['first_browser'] == list_browsers[i]),'first_browser'] = 'Others'


users.loc[(users['first_browser'] == 'Chrome Mobile') ,'first_browser'] = 'Chrome'
users.loc[(users['first_browser'] == 'Chromium'),'first_browser'] = 'Chrome'
users.loc[(users['first_browser'] == 'Mobile Firefox') ,'first_browser'] = 'Firefox'
users.loc[(users['first_browser'] == 'Mozilla'),'first_browser'] = 'Firefox'
users.loc[(users['first_browser'] == 'IE Mobile'),'first_browser'] = 'IE'
users.loc[(users['first_browser'] == 'Mobile Safari') ,'first_browser'] = 'Safari'
users.loc[(users['first_browser'] == 'Opera Mobile') ,'first_browser'] = 'Opera'
users.loc[(users['first_browser'] == 'Opera Mini'),'first_browser'] = 'Opera'

users.loc[(users['affiliate_provider'] == 'facebook-open-graph') ,'affiliate_provider'] = 'facebook'

users.loc[(users['first_device_type'] == 'Other/Unknown') ,'first_device_type'] = 'Other'
users.loc[(users['first_device_type'] == 'Android Tablet') ,'first_device_type'] = 'Tablet'
users.loc[(users['first_device_type'] == 'iPad') ,'first_device_type'] = 'Tablet'
users.loc[(users['first_device_type'] == 'Android Phone') ,'first_device_type'] = 'Phones'
users.loc[(users['first_device_type'] == 'iPhone') ,'first_device_type'] = 'Phones'
users.loc[(users['first_device_type'] == 'SmartPhone (Other)') ,'first_device_type'] = 'Phones'
users.loc[(users['first_device_type'] == 'Desktop (Other)') ,'first_device_type'] = 'Desktop'
users.loc[(users['first_device_type'] == 'Mac Desktop') ,'first_device_type'] = 'Desktop'
users.loc[(users['first_device_type'] == 'Windows Desktop') ,'first_device_type'] = 'Desktop'

users['nans'] = np.sum([
    (users['age'] == -1),
    (users['gender'] == '-unknown-'),
    (users['language'] == '-unknown-'),
    (users['first_affiliate_tracked'] == 'untracked'),
    (users['first_browser'] == '-unknown-')
], axis=0)

users['date_account_created'] = pd.to_datetime(users['date_account_created'], errors='ignore')
users['date_first_active'] = pd.to_datetime(users['timestamp_first_active'], format='%Y%m%d%H%M%S')

date_account_created = pd.DatetimeIndex(users['date_account_created'])
date_first_active = pd.DatetimeIndex(users['date_first_active'])

users['day_account_created'] = date_account_created.day
users['weekday_account_created'] = date_account_created.weekday
users['week_account_created'] = date_account_created.week
users['month_account_created'] = date_account_created.month
users['year_account_created'] = date_account_created.year
users['day_first_active'] = date_first_active.day
users['weekday_first_active'] = date_first_active.weekday
users['week_first_active'] = date_first_active.week
users['month_first_active'] = date_first_active.month
users['year_first_active'] = date_first_active.year

users['time_lag'] = (date_account_created.values - date_first_active.values).astype(int)

drop_list = [
    'date_account_created',
    'date_first_active',
    'timestamp_first_active'
]

users.drop(drop_list, axis=1, inplace=True)

sessions.rename(columns = {'user_id': 'id'}, inplace=True)

action_count = sessions.groupby(['id', 'action'])['secs_elapsed'].agg(len).unstack()
action_type_count = sessions.groupby(['id', 'action_type'])['secs_elapsed'].agg(len).unstack()
action_detail_count = sessions.groupby(['id', 'action_detail'])['secs_elapsed'].agg(len).unstack()
device_type_sum = sessions.groupby(['id', 'device_type'])['secs_elapsed'].agg(sum).unstack()

sessions_data = pd.concat([action_count, action_type_count, action_detail_count, device_type_sum],axis=1)
sessions_data.columns = sessions_data.columns.map(lambda x: str(x) + '_count')

# session device_type
list_session_device_type=['Windows Desktop','Mac Desktop','Linux Desktop','Chromebook']
for i in range(len(list_session_device_type)):
    sessions.loc[(sessions['device_type'] == list_session_device_type[i]),'device_type'] = 'Desktop'

list_session_device_type=['Android Phone','Blackberry','iPhone','iPodtouch','Opera Phone','Windows Phone']
for i in range(len(list_session_device_type)):
    sessions.loc[(sessions['device_type'] == list_session_device_type[i]),'device_type'] = 'Phones'
    
list_session_device_type=['Android App Unknown Phone/Tablet','-unknown-']
for i in range(len(list_session_device_type)):
    sessions.loc[(sessions['device_type'] == list_session_device_type[i]),'device_type'] = 'Other'

## session_action
list_session_action=['agree_terms_check','agree_terms_uncheck']
for i in range(len(list_session_action)):
    sessions.loc[(sessions['action'] == list_session_action[i]),'action'] = 'agree_terms'

list_session_action=['ajax_google_translate_description','ajax_google_translate_reviews']
for i in range(len(list_session_action)):
    sessions.loc[(sessions['action'] == list_session_action[i]),'action'] = 'google_translate'

list_session_action=['ajax_payout_edit','ajax_payout_options_by_country']
for i in range(len(list_session_action)):
    sessions.loc[(sessions['action'] == list_session_action[i]),'action'] = 'payout'
    
list_session_action=['ajax_photo_widget','ajax_photo_widget_form_iframe']
for i in range(len(list_session_action)):
    sessions.loc[(sessions['action'] == list_session_action[i]),'action'] = 'photo_widget'
    
list_session_action=['ajax_referral_banner_experiment_type','ajax_referral_banner_type']
for i in range(len(list_session_action)):
    sessions.loc[(sessions['action'] == list_session_action[i]),'action'] = 'banner'
    
list_session_action=['create','create_ach','create_multiple+create_paypal']
for i in range(len(list_session_action)):
    sessions.loc[(sessions['action'] == list_session_action[i]),'action'] = 'create'

list_session_action=['complete','complete_redirect','complete_status']
for i in range(len(list_session_action)):
    sessions.loc[(sessions['action'] == list_session_action[i]),'action'] = 'complete'    

list_session_action=['email_itinerary_colorbox','email_share','email_wishlist']
for i in range(len(list_session_action)):
    sessions.loc[(sessions['action'] == list_session_action[i]),'action'] = 'email'     

list_session_action=['home_safety_landing','home_safety_terms']
for i in range(len(list_session_action)):
    sessions.loc[(sessions['action'] == list_session_action[i]),'action'] = 'home_safety'  
    
list_session_action=['jumio_redirect','jumio_token']
for i in range(len(list_session_action)):
    sessions.loc[(sessions['action'] == list_session_action[i]),'action'] = 'jumio'   

list_session_action=['message_to_host_change','message_to_host_focus']
for i in range(len(list_session_action)):
    sessions.loc[(sessions['action'] == list_session_action[i]),'action'] = 'message_to_host'  

list_session_action=['qt_reply_v2','qt_with','qt2']
for i in range(len(list_session_action)):
    sessions.loc[(sessions['action'] == list_session_action[i]),'action'] = 'qt' 

list_session_action=['update_cached','update_friends_display','update_hide_from_search_engines','update_notifications']
for i in range(len(list_session_action)):
    sessions.loc[(sessions['action'] == list_session_action[i]),'action'] = 'update' 

list_session_action=['travel_plans_current','travel_plans_previous']
for i in range(len(list_session_action)):
    sessions.loc[(sessions['action'] == list_session_action[i]),'action'] = 'travel_plans'     

sessions.loc[(sessions['action'] == 'friends_new'),'action'] = 'friends' 
sessions.loc[(sessions['action'] == 'departments'),'action'] = 'department'
sessions.loc[(sessions['action'] == 'transaction_history_paginated'),'action'] = 'transaction_history'
sessions.loc[(sessions['action'] == 'edit_verification'),'action'] = 'edit'
sessions.loc[(sessions['action'] == 'kba_update'),'action'] = 'kba'   
sessions.loc[(sessions['action'] == 'listings'),'action'] = 'listing'   
sessions.loc[(sessions['action'] == 'multi_message'),'action'] = 'multi'  
sessions.loc[(sessions['action'] == 'reviews_new'),'action'] = 'reviews'  
sessions.loc[(sessions['action'] == 'similar_listings_v2'),'action'] = 'similar_listings' 
sessions.loc[(sessions['action'] == 'show_code'),'action'] = 'show' 
sessions.loc[(sessions['action'] == 'social_connections'),'action'] = 'social' 
sessions.loc[(sessions['action'] == 'terms_and_conditions'),'action'] = 'terms' 

#action_detail

list_coupon_session_action_detail=['apply_coupon','apply_coupon_click','apply_coupon_click_success','apply_coupon_error','coupon_code_click','coupon_field_focus']
for i in range(len(list_coupon_session_action_detail)):
    sessions.loc[(sessions['action_detail'] == list_coupon_session_action_detail[i]),'sessions_action_detail'] = 'coupon'

list_cancel_session_action_detail=['cancellation_policies','cancellation_policy','cancellation_policy_click']
for i in range(len(list_cancel_session_action_detail)):
    sessions.loc[(sessions['action_detail'] == list_cancel_session_action_detail[i]),'sessions_action_detail'] = 'cancel'

list_guest_session_action_detail=['guest_cancellation','guest_itinerary','guest_receipt']
for i in range(len(list_guest_session_action_detail)):
    sessions.loc[(sessions['action_detail'] == list_guest_session_action_detail[i]),'sessions_action_detail'] = 'guest'
    
list_host_session_action_detail=['host_guarantee','host_home','host_refund_guest','host_respond','host_respond_page','host_standard_suspension']
for i in range(len(list_host_session_action_detail)):
    sessions.loc[(sessions['action_detail'] == list_host_session_action_detail[i]),'sessions_action_detail'] = 'host'    
    
list_listing_session_action_detail=['listing_descriptions','listing_recommendations','listing_reviews','listing_reviews_page','manage_listing','view_listing']
for i in range(len(list_listing_session_action_detail)):
    sessions.loc[(sessions['action_detail'] == list_listing_session_action_detail[i]),'sessions_action_detail'] = 'listing'       

     
list_message_session_action_detail=['lookup_message_thread','message_inbox','message_post','message_thread','message_to_host_change','message_to_host_focus']
for i in range(len(list_message_session_action_detail)):
    sessions.loc[(sessions['action_detail'] == list_message_session_action_detail[i]),'sessions_action_detail'] = 'message'       


# Most used device
sessions_data['most_used_device'] = sessions.groupby('id')['device_type'].max()

users = users.join(sessions_data, on='id')

secs_elapsed = sessions.groupby('id')['secs_elapsed']

secs_elapsed = secs_elapsed.agg(
    {
        'secs_elapsed_sum': np.sum,
        'secs_elapsed_mean': np.mean,
        'secs_elapsed_min': np.min,
        'secs_elapsed_max': np.max,
        'secs_elapsed_median': np.median,
        'secs_elapsed_std': np.std,
        'secs_elapsed_var': np.var,
        'day_pauses': lambda x: (x > 86400).sum(),
        'long_pauses': lambda x: (x > 300000).sum(),
        'short_pauses': lambda x: (x < 3600).sum(),
        'session_length' : np.count_nonzero
    }
)

users = users.join(secs_elapsed, on='id')


categorical_features = [
    'gender', 'signup_method', 'signup_flow', 'language',
    'affiliate_channel', 'affiliate_provider', 'first_affiliate_tracked',
    'signup_app', 'first_device_type', 'first_browser', 'most_used_device'
]
users = pd.get_dummies(users, columns=categorical_features)


users.set_index('id', inplace=True)
users.loc[train_users['id']].to_csv('train4_users.csv')
users.loc[test_users['id']].drop('country_destination', axis=1).to_csv('test4_users.csv')


/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:221: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version


In [3]:
train_users = pd.read_csv('train4_users.csv')

train_users.fillna(-1, inplace=True)
y_train = train_users['country_destination']
train_users.drop(['country_destination', 'id'], axis=1, inplace=True)
x_train = train_users.values

label_encoder = LabelEncoder()
encoded_y_train = label_encoder.fit_transform(y_train)

In [14]:
n = len(x_train);n

213451

In [15]:
val_idxs = get_cv_idxs(n)

In [ ]:
cat_vars = [1,2,3]

In [16]:
md = ColumnarModelData.from_data_frame(PATH,val_idxs,x_train,encoded_y_train,cat_flds = categorical_features,bs=128)

KeyError: "['gender' 'signup_method' 'signup_flow' 'language' 'affiliate_channel' 'affiliate_provider'\n 'first_affiliate_tracked' 'signup_app' 'first_device_type' 'first_browser' 'most_used_device'] not in index"

In [ ]:
cat_sz = [(c, len(x[c].cat.categories)+1) for c in cat_vars]

In [ ]:
m = md.get_learner()